In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder        # for label encoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
seed=7
np.random.seed(seed)

In [3]:
dataframe=pd.read_csv('sonar.csv',header=None)

In [4]:
dataset=dataframe.values
dataset.shape

(208, 61)

In [5]:
x=dataset[:,0:60].astype('float32')
x

array([[0.02  , 0.0371, 0.0428, ..., 0.0084, 0.009 , 0.0032],
       [0.0453, 0.0523, 0.0843, ..., 0.0049, 0.0052, 0.0044],
       [0.0262, 0.0582, 0.1099, ..., 0.0164, 0.0095, 0.0078],
       ...,
       [0.0522, 0.0437, 0.018 , ..., 0.0138, 0.0077, 0.0031],
       [0.0303, 0.0353, 0.049 , ..., 0.0079, 0.0036, 0.0048],
       [0.026 , 0.0363, 0.0136, ..., 0.0036, 0.0061, 0.0115]],
      dtype=float32)

In [6]:
y=dataset[:,60]
y

array(['R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R',
       'R', 'R', 'R', 'R', 'R', 'R', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M',
       'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M', 'M

In [7]:
le=LabelEncoder()
le.fit(y)

LabelEncoder()

In [8]:
le.classes_

array(['M', 'R'], dtype=object)

In [9]:
encoded_y=le.transform(y)

In [10]:
encoded_y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [11]:
def create_baseline():
    model=Sequential()
    model.add(Dense(60,activation='relu',input_shape=(60,)))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['acc'])
    return model

In [12]:
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, x, encoded_y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 84.18% (6.08%)


In [13]:
# evaluate baseline model with standardized dataset

In [14]:
np.random.seed(seed)

In [15]:
estimator=[]
estimator.append(('standardize',StandardScaler()))
estimator.append(('mlp',KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline=Pipeline(estimator)
kfold=StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
result=cross_val_score(pipeline,x,encoded_y,cv=kfold)

In [16]:
print("Standardized: %.2f%% (%.2f%%)" % (result.mean()*100, result.std()*100))

Standardized: 85.59% (7.46%)


# Smaller Model

In [18]:
def create_smaller():
    model=Sequential()
    model.add(Dense(30,activation='relu',input_shape=(60,)))
    model.add(Dense(1,activation='sigmoid'))
    
    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['acc'])
    return model

In [19]:
estimator=KerasClassifier(build_fn=create_smaller,epochs=100,batch_size=5,verbose=0)
kfold=StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
result=cross_val_score(estimator,x,encoded_y,cv=kfold)
print(result.mean()*100,result.std()*100)

83.70562846042635 7.463458112674119


In [20]:
print("Results: %.2f%% (%.2f%%)" % (result.mean()*100, result.std()*100))

Results: 83.71% (7.46%)


In [21]:
# through pipeline standardize data
np.random.seed(seed)
estimator=[]
estimator.append(('standardize',StandardScaler()))
estimator.append(('mlp',KerasClassifier(build_fn=create_smaller,epochs=100,batch_size=5,verbose=0)))
pipeline=Pipeline(estimator)
kfold=StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
result=cross_val_score(pipeline,x,encoded_y,cv=kfold)

In [22]:
print("Standardized: %.2f%% (%.2f%%)" % (result.mean()*100, result.std()*100))

Standardized: 84.59% (7.99%)


# Larger Model

In [23]:
def create_Larger():
    model=Sequential()
    model.add(Dense(60,activation='relu',input_shape=(60,)))
    model.add(Dense(30,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    
    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['acc'])
    return model

In [24]:
estimator = KerasClassifier(build_fn=create_Larger,epochs=100,batch_size=5,verbose=0)
kfold=StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
result=cross_val_score(estimator,x,encoded_y,cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (result.mean()*100, result.std()*100))

Results: 86.59% (7.59%)


In [25]:
# Standardize data

In [26]:
np.random.seed(seed)
estimator=[]
estimator.append(('Standardize',StandardScaler()))
estimator.append(('mlp',KerasClassifier(build_fn=create_Larger,epochs=100,batch_size=5,verbose=0)))
pipeline=Pipeline(estimator)
kfold=StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
result=cross_val_score(pipeline,x,encoded_y,cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (result.mean()*100, result.std()*100))

Standardized: 84.61% (6.36%)


In [27]:
print("Larger: %.2f%% (%.2f%%)" % (result.mean()*100, result.std()*100))

Larger: 84.61% (6.36%)


# Fully Scaled Model

In [28]:
def fully_scaled():
    model=Sequential()
    model.add(Dense(512,activation='relu',input_shape=(60,)))
    model.add(Dense(256,activation='relu'))
    model.add(Dense(128,activation='relu'))
    model.add(Dense(64,activation='relu'))
    model.add(Dense(32,activation='relu'))
    model.add(Dense(16,activation='relu'))
    model.add(Dense(4,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    
    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['acc'])
    return model

estimator=[]
estimator.append(('Standardize',StandardScaler()))
estimator.append(('mlp',KerasClassifier(build_fn=fully_scaled,epochs=200,batch_size=128,verbose=0)))
pipeline=Pipeline(estimator)
kfold=StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)


In [29]:
result=cross_val_score(pipeline,x,encoded_y,cv=kfold)
print("Fully Scaled: %.2f%% (%.2f%%)" % (result.mean()*100, result.std()*100))

Fully Scaled: 85.54% (6.95%)


# Tuned Model

In [30]:
def Tuned():
    model=Sequential()
    model.add(Dense(512,activation='relu',input_shape=(60,)))
    model.add(Dense(256,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    
    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics = ['accuracy'])
    return model

estimator=[]
estimator.append(('Standardize',StandardScaler()))
estimator.append(('mlp',KerasClassifier(build_fn=Tuned,epochs=100,batch_size=128,verbose=0)))
pipeline=Pipeline(estimator)
kfold=StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
result=cross_val_score(pipeline,x,encoded_y,cv=kfold)
print("Tuned Model: %.2f%% (%.2f%%)" % (result.mean()*100, result.std()*100))

Tuned Model: 86.56% (8.27%)


# Functional API

In [12]:
import keras
from keras import layers

In [32]:
def baseline_model():
    inputs = keras.Input(shape=(60,))
    x=layers.Dense(60,activation='relu')(inputs)
    outputs=layers.Dense(1,activation='sigmoid')(x)
    model=keras.Model(inputs,outputs)
    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics = ['accuracy'])
    
    return model

In [33]:
#through pipeline standardize data
np.random.seed(seed)
estimator=[]
estimator.append(('standardize',StandardScaler()))
estimator.append(('mlp',KerasClassifier(build_fn=baseline_model,epochs=100,batch_size=5,verbose=0)))
pipeline=Pipeline(estimator)
kfold=StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
result=cross_val_score(pipeline,x,encoded_y,cv=kfold)
print("Result : %.2f%% (%.2f%%)" % (result.mean()*100, result.std()*100))

Result : 85.59% (7.46%)


In [34]:
def create_small():
    inputs = keras.Input(shape=(60,))
    x=layers.Dense(30,activation='relu')(inputs)
    outputs=layers.Dense(1,activation='sigmoid')(x)
    model=keras.Model(inputs,outputs)
    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics = ['accuracy'])
    
    return model

In [35]:
#through pipeline standardize data
np.random.seed(seed)
estimator=[]
estimator.append(('standardize',StandardScaler()))
estimator.append(('mlp',KerasClassifier(build_fn=create_small,epochs=100,batch_size=5,verbose=0)))
pipeline=Pipeline(estimator)
kfold=StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
result=cross_val_score(pipeline,x,encoded_y,cv=kfold)
print("Small : %.2f%% (%.2f%%)" % (result.mean()*100, result.std()*100))

Small : 85.06% (7.61%)


In [36]:
def create_Larger():
    inputs = keras.Input(shape=(60,))
    x=layers.Dense(60,activation='relu')(inputs)
    x=layers.Dense(30,activation='relu')(x)
    outputs=layers.Dense(1,activation='sigmoid')(x)
    
    model=keras.Model(inputs,outputs)
    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics = ['accuracy'])
    
    return model

In [37]:
#through pipeline standardize data
np.random.seed(seed)
estimator=[]
estimator.append(('standardize',StandardScaler()))
estimator.append(('mlp',KerasClassifier(build_fn=create_Larger,epochs=100,batch_size=5,verbose=0)))
pipeline=Pipeline(estimator)
kfold=StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
result=cross_val_score(pipeline,x,encoded_y,cv=kfold)
print("Larger : %.2f%% (%.2f%%)" % (result.mean()*100, result.std()*100))

Larger : 85.09% (5.87%)


In [38]:
def fully_scaled():
    inputs = keras.Input(shape=(60,))
    x=layers.Dense(512,activation='relu')(inputs)
    x=layers.Dense(256,activation='relu')(x)
    x=layers.Dense(128,activation='relu')(x)
    x=layers.Dense(64,activation='relu')(x)
    x=layers.Dense(32,activation='relu')(x)
    x=layers.Dense(16,activation='relu')(x)
    x=layers.Dense(4,activation='relu')(x)
    outputs=layers.Dense(1,activation='sigmoid')(x)
    
    model=keras.Model(inputs,outputs)
    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics = ['accuracy'])
    
    return model

#through pipeline standardize data
np.random.seed(seed)
estimator=[]
estimator.append(('standardize',StandardScaler()))
estimator.append(('mlp',KerasClassifier(build_fn=fully_scaled,epochs=100,batch_size=5,verbose=0)))
pipeline=Pipeline(estimator)
kfold=StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
result=cross_val_score(pipeline,x,encoded_y,cv=kfold)
print("Fully_scaled : %.2f%% (%.2f%%)" % (result.mean()*100, result.std()*100))

Fully_scaled : 82.66% (9.22%)


In [39]:
def create_tuned():
    inputs = keras.Input(shape=(60,))
    x=layers.Dense(90,activation='relu')(inputs)
    x=layers.Dense(60,activation='relu')(x)
    outputs=layers.Dense(1,activation='sigmoid')(x)
    
    model=keras.Model(inputs,outputs)
    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics = ['accuracy'])
    
    return model



In [40]:
#through pipeline standardize data
np.random.seed(seed)
estimator=[]
estimator.append(('standardize',StandardScaler()))
estimator.append(('mlp',KerasClassifier(build_fn=create_tuned,epochs=100,batch_size=5,verbose=0)))
pipeline=Pipeline(estimator)
kfold=StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
result=cross_val_score(pipeline,x,encoded_y,cv=kfold)
print("Tuned : %.2f%% (%.2f%%)" % (result.mean()*100, result.std()*100))

Tuned : 82.61% (7.74%)


# Model Sub CLassing

In [41]:
import keras
from keras import layers

In [42]:
class baseline(keras.Model):
     def __init__(self):
            super(baseline,self).__init__()
            self.dense1=layers.Dense(60,activation='relu')
            self.dense2=layers.Dense(1,activation='sigmoid')
     def call(self,inputs):
          x=self.dense1(inputs)
          return self.dense2(x)
def create_baseline():
    inputs=keras.Input(shape=(60,))
    Mymodel=baseline()
    outputs=Mymodel.call(inputs)
    
    model=keras.Model(inputs,outputs)
    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['acc'])
    return model

    
#through pipeline standardize data
np.random.seed(seed)
estimator=[]
estimator.append(('standardize',StandardScaler()))
estimator.append(('mlp',KerasClassifier(build_fn=create_baseline,epochs=100,batch_size=5,verbose=0)))
pipeline=Pipeline(estimator)
kfold=StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
result=cross_val_score(pipeline,x,encoded_y,cv=kfold)
print("Result : %.2f%% (%.2f%%)" % (result.mean()*100, result.std()*100))

Result : 85.59% (7.46%)


In [43]:
class small(keras.Model):
     def __init__(self):
            super(small,self).__init__()
            self.dense1=layers.Dense(30,activation='relu')
            self.dense2=layers.Dense(1,activation='sigmoid')
     def call(self,inputs):
          x=self.dense1(inputs)
          return self.dense2(x)
def create_small():
    inputs=keras.Input(shape=(60,))
    Mymodel=small()
    outputs=Mymodel.call(inputs)
    
    model=keras.Model(inputs,outputs)
    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['acc'])
    return model

    
#through pipeline standardize data
np.random.seed(seed)
estimator=[]
estimator.append(('standardize',StandardScaler()))
estimator.append(('mlp',KerasClassifier(build_fn=create_small,epochs=100,batch_size=5,verbose=0)))
pipeline=Pipeline(estimator)
kfold=StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
result=cross_val_score(pipeline,x,encoded_y,cv=kfold)
print("small : %.2f%% (%.2f%%)" % (result.mean()*100, result.std()*100))

small : 85.06% (7.61%)


In [44]:
class Larger(keras.Model):
     def __init__(self):
            super(Larger,self).__init__()
            self.dense1=layers.Dense(60,activation='relu')
            self.dense2=layers.Dense(30,activation='relu')
            self.dense3=layers.Dense(1,activation='sigmoid')
     def call(self,inputs):
          x=self.dense1(inputs)
          x=self.dense2(x)
          return self.dense3(x)
def create_large():
    inputs=keras.Input(shape=(60,))
    Mymodel=Larger()
    outputs=Mymodel.call(inputs)
    
    model=keras.Model(inputs,outputs)
    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['acc'])
    return model

    
#through pipeline standardize data
np.random.seed(seed)
estimator=[]
estimator.append(('standardize',StandardScaler()))
estimator.append(('mlp',KerasClassifier(build_fn=create_large,epochs=100,batch_size=5,verbose=0)))
pipeline=Pipeline(estimator)
kfold=StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
result=cross_val_score(pipeline,x,encoded_y,cv=kfold)
print("Large : %.2f%% (%.2f%%)" % (result.mean()*100, result.std()*100))

Large : 84.11% (6.11%)


In [13]:
class FullScaled(keras.Model):
     def __init__(self):
            super(FullScaled,self).__init__()
            self.dense1=layers.Dense(512,activation='relu')
            self.dense2=layers.Dense(256,activation='relu')
            self.dense3=layers.Dense(128,activation='relu')
            self.dense4=layers.Dense(64,activation='relu')
            self.dense5=layers.Dense(32,activation='relu')
            self.dense6=layers.Dense(16,activation='relu')
            self.dense7=layers.Dense(4,activation='relu')
            self.dense8=layers.Dense(1,activation='sigmoid')
     def call(self,inputs):
          x=self.dense1(inputs)
          x=self.dense2(x)
          x=self.dense3(x)
          x=self.dense4(x)
          x=self.dense5(x)
          x=self.dense6(x)
          x=self.dense7(x)
          return self.dense8(x)
def create_fullScaled():
    inputs=keras.Input(shape=(60,))
    Mymodel=FullScaled()
    outputs=Mymodel.call(inputs)
    
    model=keras.Model(inputs,outputs)
    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['acc'])
    return model

    
#through pipeline standardize data
np.random.seed(seed)
estimator=[]
estimator.append(('standardize',StandardScaler()))
estimator.append(('mlp',KerasClassifier(build_fn=create_fullScaled,epochs=300,batch_size=5,verbose=0)))
pipeline=Pipeline(estimator)
kfold=StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
result=cross_val_score(pipeline,x,encoded_y,cv=kfold)
print("Full_scaled : %.2f%% (%.2f%%)" % (result.mean()*100, result.std()*100))

Full_scaled : 85.56% (7.40%)


In [15]:
class Tuned(keras.Model):
     def __init__(self):
            super(Tuned,self).__init__()
            self.dense1=layers.Dense(90,activation='relu')
            self.dense2=layers.Dense(60,activation='relu')
            self.dense3=layers.Dense(1,activation='sigmoid')
     def call(self,inputs):
          x=self.dense1(inputs)
          x=self.dense2(x)
          return self.dense3(x)
def create_tuned():
    inputs=keras.Input(shape=(60,))
    Mymodel=Tuned()
    outputs=Mymodel.call(inputs)
    
    model=keras.Model(inputs,outputs)
    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['acc'])
    return model

    
#through pipeline standardize data
np.random.seed(seed)
estimator=[]
estimator.append(('standardize',StandardScaler()))
estimator.append(('mlp',KerasClassifier(build_fn=create_tuned,epochs=100,batch_size=5,verbose=0)))
pipeline=Pipeline(estimator)
kfold=StratifiedKFold(n_splits=10,shuffle=True,random_state=seed)
result=cross_val_score(pipeline,x,encoded_y,cv=kfold)
print("Full_scaled : %.2f%% (%.2f%%)" % (result.mean()*100, result.std()*100))

Full_scaled : 82.13% (8.25%)


# Without Scikit Learn

In [ ]:
from keras import models
from keras import layers

In [23]:
dataset

array([[0.02, 0.0371, 0.0428, ..., 0.009, 0.0032, 'R'],
       [0.0453, 0.0523, 0.0843, ..., 0.0052, 0.0044, 'R'],
       [0.0262, 0.0582, 0.1099, ..., 0.0095, 0.0078, 'R'],
       ...,
       [0.0522, 0.0437, 0.018, ..., 0.0077, 0.0031, 'M'],
       [0.0303, 0.0353, 0.049, ..., 0.0036, 0.0048, 'M'],
       [0.026, 0.0363, 0.0136, ..., 0.0061, 0.0115, 'M']], dtype=object)

In [45]:
def get_model():
    model=Sequential()
    model.add(Dense(512,activation='relu',input_shape=(60,)))
    model.add(Dense(256,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['acc'])
    return model

In [43]:
k=10
num_val_samples = len(x) // k
np.random.shuffle(x)
all_scores = []

In [46]:
for i in range(k):
    print('processing fold # ', i+1)
    val_data = x[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = encoded_y[i * num_val_samples: (i + 1) * num_val_samples]
  
    partial_train_data = np.concatenate(                    
      [x[:i * num_val_samples],
      x[(i + 1 ) * num_val_samples:]],
    axis = 0)
    partial_train_targets = np.concatenate(
      [encoded_y[:i * num_val_samples],
      encoded_y[(i + 1 ) * num_val_samples:]],
    axis = 0)
    model = get_model()
    history = model.fit(partial_train_data, partial_train_targets, epochs = num_epochs, batch_size = 5, verbose = 0)
    val_binary_crossentropy, val_adam = model.evaluate(val_data, val_targets, verbose = 0)
    all_scores.append(val_adam)
    print("K-Fold Witout Scikit-learn : (Accuracy)Mean %.2f%% (Error)Std (%.2f%%)" % (np.mean(all_scores)*100, np.std(all_scores)*100))

processing fold #  1
K-Fold Witout Scikit-learn : (Accuracy)Mean 49.09% (Error)Std (12.03%)
processing fold #  2
K-Fold Witout Scikit-learn : (Accuracy)Mean 49.58% (Error)Std (11.63%)
processing fold #  3
K-Fold Witout Scikit-learn : (Accuracy)Mean 49.62% (Error)Std (11.17%)
processing fold #  4
K-Fold Witout Scikit-learn : (Accuracy)Mean 48.93% (Error)Std (11.05%)
processing fold #  5
K-Fold Witout Scikit-learn : (Accuracy)Mean 49.00% (Error)Std (10.68%)
processing fold #  6
K-Fold Witout Scikit-learn : (Accuracy)Mean 50.62% (Error)Std (12.10%)
processing fold #  7
K-Fold Witout Scikit-learn : (Accuracy)Mean 50.29% (Error)Std (11.82%)
processing fold #  8
K-Fold Witout Scikit-learn : (Accuracy)Mean 49.44% (Error)Std (12.01%)
processing fold #  9
K-Fold Witout Scikit-learn : (Accuracy)Mean 49.47% (Error)Std (11.69%)
processing fold #  10
K-Fold Witout Scikit-learn : (Accuracy)Mean 49.25% (Error)Std (11.43%)
